# 필요 라이브러리 설치

In [1]:
!pip install git+https://github.com/huggingface/transformers
# !pip install -U transformers
!pip install peft
!pip install datasets
!pip install fire
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4_cq9q_u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4_cq9q_u
  Resolved https://github.com/huggingface/transformers to commit 95b374952dc27d8511541d6f5a4e22c9ec11fb24
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for safetensors>=0.3.1 from https://files.pythonhosted.org/packages/9c/ee/de72fa290bef49b22aacdf112e70f40d3b1beb526dd65e94e0f540d192f0/safetensors-0.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7640036 sha256=710aa59567d9118c7ecc895a3bc

# 라이브러리 불러오기

In [1]:
import os
import sys
import textwrap
from typing import List

# import fire
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
# import pandas as pd
import transformers
from transformers import (AutoModelForCausalLM, AutoTokenizer, default_data_collator, AutoTokenizer,
                          get_linear_schedule_with_warmup, LlamaTokenizer, LlamaForCausalLM)
from datasets import load_dataset, Dataset, DatasetDict
from peft import (get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig,
                  TaskType, PeftType, prepare_model_for_int8_training, LoraConfig,
                  get_peft_model_state_dict)


/home/work/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 로드

In [2]:
import pickle

with open(file='/home/work/data00/dataset/llama2/test.pickle', mode='rb') as f:
    test_data=pickle.load(f)

In [3]:
test_data['output']

['5년간',
 '매년 3월 말이나 4월 초에 열리는 볼로냐아동도서전 현장',
 'no',
 '씨엔블루',
 '형민우',
 '한국교통대학교',
 '러시아연방은 보편적으로 인정되는 국제법 규범과 러시아연방이 체결한 국제조약에 의해 토착 소수민족의 권리를 보장한다.',
 'no',
 '법률',
 '엑스포',
 '데이브 머스테인',
 '257명',
 '군주, 상원, 그리고 하원',
 '2007년 4월 2일',
 '통일민주당',
 '681년',
 '연방의 모든 주와 지역의 법원, 판사 및 국민',
 'yes',
 '히가시노 게이고, 프란츠 카프카',
 'no',
 'yes',
 'no',
 '식민 지배의 잔해',
 '두바이 국제공항',
 '수령한 날로부터 5일 이내에 제정 또는 개정된 기본법에 서명하고, 관보에 게재할 것을 명하여야 한다',
 '헌법',
 '스누라이프',
 "아랍어로 '문'을 뜻하는 '밥'이라는 말",
 '1929년',
 '가와사키 프론탈레',
 '하원이 동의하지 않는 해당 법률안의 수정안',
 '이유를 기재한 법원 명령',
 '56개',
 '국제조약',
 '1889년',
 '해당 증언을 거부',
 '임면·교육·심사·상벌',
 '정부 또는 정부위원',
 '전략기획실',
 '일반 판사',
 '대통령',
 'no',
 'yes',
 '데이터베이스',
 '마우리치오 사리',
 '서울특별시',
 '밤12시부터 오전6시까지',
 '합법적인 상위 당국자가 공포한 명령 또는 지시',
 '국민',
 '프랑스',
 'KBS2',
 '법률',
 '무파사',
 '재단',
 '법률로 정당화되는 경우',
 '스콧 피츠제럴드',
 'no',
 '가족 생활급과 수입에 대한 가족의 권리',
 '종교를 믿지 않는 공민',
 '사적 통신은 침해되어서는 아니 된다.',
 '남부에 대해서만 관대했던 논조가 반감을 불러 일으켜',
 'no',
 '1623년',
 '1936년 5월 2일 ~ )은 대한민국의 영화 감독, 영화 각본가, 영화 제작자이다',
 '마리아',
 'no',
 '환상속의

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 64
learning_rate = 3e-2
num_epochs = 5
batch_size = 8

LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 16
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 30
OUTPUT_DIR = "/home/work/data00/result/experiences"

DEVICE = 'cuda'

# 모델 불러오기

In [5]:
BASE_MODEL = "beomi/llama-2-ko-7b"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto", # auto -> cuda:0 ## memory error 해결
)

print('finish')

tokenizer = AutoTokenizer.from_pretrained("beomi/llama-2-ko-7b")

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 15/15 [00:56<00:00,  3.77s/it]


finish


TypeError: not a string

# 성능 체크

In [6]:
from transformers import AutoTokenizer, LlamaForCausalLM, GenerationConfig
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap

In [7]:
def create_prompt(data_point):
    return f"""아래는 작업을 설명하는 명령어로, 함께 입력되는 입력 문맥 및 답변과 쌍을 이룹니다. 주어진 문맥에 맞게 적절한 응답을 작성하세요.
### 명령어:
{data_point["instruction"]}
### 입력 문맥:
{data_point["input"]}
### Response:
"""

In [36]:
def generate_response(prompt: str, model: model):
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)

    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=16, # test data의 평균 output길이 = 10
        )

def format_response(response) -> str:
    # print(response.sequences)
    decoded_output = tokenizer.decode(response.sequences[0])
    # print(decoded_output)
    response = decoded_output.split("### Response:")[1].strip()
    # print(response)
    return "\n".join(textwrap.wrap(response))

def ask_alpaca(prompt, model: model) -> str:
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    response = format_response(response)
    return response

In [38]:
from tqdm import tqdm
DEVICE = 'cuda'
a = []
i = 0
for inst in tqdm(test_data):
    # print(inst)

    response = ask_alpaca(inst, model)

    # print(response)
    # if (response == "yes" and inst['is_first_in'] ==1) or (response == "no" and inst['is_first_in'] ==0) or (inst['party'] == "Complainant" and response == "yes"):
    #     a.append((inst['ID'], 1))
    # else:
    #     a.append((inst['ID'], 0))
    a.append((i, response))

    i += 1

 36%|███▋      | 3456/9475 [3:20:20<6:01:17,  3.60s/it]

# 결과 확인

In [ ]:
import pandas as pd

tmp_result = pd.DataFrame(a)

tmp_result.to_csv('/home/work/data00/result/no_finetuned_result_01.csv')

# 평가

In [15]:
!pip install absl-py
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import evaluate

/home/work/anaconda3/envs/llama2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metric = evaluate.load('rouge')

In [4]:
import pickle

with open(file='/home/work/data00/dataset/llama2/test.pickle', mode='rb') as f:
    test_data=pickle.load(f)

with open(file='/home/work/data00/result/no_finetune.pickle', mode='rb') as f:
    no_finetuned = pickle.load(f)    

In [7]:
references = test_data['output']

In [11]:
predictions = list(no_finetuned[1])

In [12]:
metric.compute(predictions=predictions, references=references)

{'rouge1': 0.059141695888397725,
 'rouge2': 0.014759804253207946,
 'rougeL': 0.05910112051536849,
 'rougeLsum': 0.05926589300467926}

# after finetuning

In [1]:
import evaluate

metric = evaluate.load('rouge')

/home/work/anaconda3/envs/llama2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pickle

with open(file='/home/work/data00/dataset/llama2/test.pickle', mode='rb') as f:
    test_data=pickle.load(f)

with open(file='/home/work/data00/result/finetune.pickle', mode='rb') as f:
    finetuned = pickle.load(f)    

In [3]:
references = test_data['output']
predictions = list(finetuned[1])

In [4]:
metric.compute(predictions=predictions, references=references)

{'rouge1': 0.06441145339562218,
 'rouge2': 0.016158021398126943,
 'rougeL': 0.06426508964503681,
 'rougeLsum': 0.06445923235369135}